# Scraping Quotes 
## Test the scraping script part by part

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [111]:
# URL of the website to scrape
url_base = "http://quotes.toscrape.com/"

In [112]:
# List to store scraped data
list_quote = []

In [113]:
# Test response
response = requests.get(url_base)
response.status_code

200

In [114]:
soup = BeautifulSoup(response.text, 'html.parser')
soup

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<title>Quotes to Scrape</title>
<link href="/static/bootstrap.min.css" rel="stylesheet"/>
<link href="/static/main.css" rel="stylesheet"/>
</head>
<body>
<div class="container">
<div class="row header-box">
<div class="col-md-8">
<h1>
<a href="/" style="text-decoration: none">Quotes to Scrape</a>
</h1>
</div>
<div class="col-md-4">
<p>
<a href="/login">Login</a>
</p>
</div>
</div>
<div class="row">
<div class="col-md-8">
<div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
<span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>
<span>by <small class="author" itemprop="author">Albert Einstein</small>
<a href="/author/Albert-Einstein">(about)</a>
</span>
<div class="tags">
            Tags:
            <meta class="keywords" content="change,deep-thoughts,thinking,world" itemprop="keywords"/>
<a class="

In [115]:
quotes = soup.find_all('div', class_='quote')
quotes

[<div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
 <span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>
 <span>by <small class="author" itemprop="author">Albert Einstein</small>
 <a href="/author/Albert-Einstein">(about)</a>
 </span>
 <div class="tags">
             Tags:
             <meta class="keywords" content="change,deep-thoughts,thinking,world" itemprop="keywords"/>
 <a class="tag" href="/tag/change/page/1/">change</a>
 <a class="tag" href="/tag/deep-thoughts/page/1/">deep-thoughts</a>
 <a class="tag" href="/tag/thinking/page/1/">thinking</a>
 <a class="tag" href="/tag/world/page/1/">world</a>
 </div>
 </div>,
 <div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
 <span class="text" itemprop="text">“It is our choices, Harry, that show what we truly are, far more than our abilities.”</span>
 <span>by <small class="author" itempr

In [116]:
test_list = []
for quote in quotes:
    text = quote.find('span', class_='text').get_text()
    author = quote.find('small', class_='author').get_text()
    # for tag in quote.find_all('a', class_='tag'):
    #     print(tag.get_text())
    tags = [tag.get_text() for tag in quote.find_all('a', class_='tag')]
    # print(text)
    # print(author)
    # print(tags)

    test_list.append({
        'quote': text,
        'author': author,
        'tags': ', '.join(tags)
    })
# test_list
pd.DataFrame(test_list)

,quote,author,tags
0,“The world as we have created it is a process ...,Albert Einstein,"change, deep-thoughts, thinking, world"
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"abilities, choices"
2,“There are only two ways to live your life. On...,Albert Einstein,"inspirational, life, live, miracle, miracles"
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,"aliteracy, books, classic, humor"
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,"be-yourself, inspirational"
5,“Try not to become a man of success. Rather be...,Albert Einstein,"adulthood, success, value"
6,“It is better to be hated for what you are tha...,André Gide,"life, love"
7,"“I have not failed. I've just found 10,000 way...",Thomas A. Edison,"edison, failure, inspirational, paraphrased"
8,“A woman is like a tea bag; you never know how...,Eleanor Roosevelt,misattributed-eleanor-roosevelt
9,"“A day without sunshine is like, you know, nig...",Steve Martin,"humor, obvious, simile"


In [117]:
test_next_page = soup.find('li', class_='next')
test_next_page

<li class="next">
<a href="/page/2/">Next <span aria-hidden="true">→</span></a>
</li>

In [118]:
test_next_page.find('a')['href']

'/page/2/'

In [119]:
url_base + test_next_page.find('a')['href']

'http://quotes.toscrape.com//page/2/'

In [120]:
requests.get(url_base + test_next_page.find('a')['href']).status_code

200

In [122]:
# Set a function to scrape the data from the URL above
def scrape_quote(url):
    response = requests.get(url)
    print("=== Get response from the web ===")
    if response.status_code == 200:
        print("=== Status code 200 ===")
        soup = BeautifulSoup(response.text, 'html.parser')

        # Retrieve quotes
        quotes = soup.find_all('div', class_='quote')
        print("=== Start scraping ===")
        for quote in quotes:
            text = quote.find('span', class_='text').get_text()
            author = quote.find('small', class_='author').get_text()
            tags = [tag.get_text() for tag in quote.find_all('a', class_='tag')]
            list_quote.append({
                'quote': text,
                'author': author,
                'tags': ', '.join(tags)
            })
        print("=== Done scraping ===")

        # Find the next page link
        next_page = soup.find('li', class_='next')
        if next_page:
            next_url = url_base + next_page.find('a')['href']
            print("=== Move to the next page ===")
            scrape_quote(next_url)

    else:
        print(f"=== Failed to retrieve page: {url} ===")

In [123]:
# Start scraping by running the function
scrape_quote(url_base)

=== Get response from the web ===
=== Status code 200 ===
=== Start scraping ===
=== Done scraping ===
=== Move to the next page ===
=== Get response from the web ===
=== Status code 200 ===
=== Start scraping ===
=== Done scraping ===
=== Move to the next page ===
=== Get response from the web ===
=== Status code 200 ===
=== Start scraping ===
=== Done scraping ===
=== Move to the next page ===
=== Get response from the web ===
=== Status code 200 ===
=== Start scraping ===
=== Done scraping ===
=== Move to the next page ===
=== Get response from the web ===
=== Status code 200 ===
=== Start scraping ===
=== Done scraping ===
=== Move to the next page ===
=== Get response from the web ===
=== Status code 200 ===
=== Start scraping ===
=== Done scraping ===
=== Move to the next page ===
=== Get response from the web ===
=== Status code 200 ===
=== Start scraping ===
=== Done scraping ===
=== Move to the next page ===
=== Get response from the web ===
=== Status code 200 ===
=== Start s

In [124]:
# Save the data into a dataframe
df = pd.DataFrame(list_quote)

In [125]:
df.head()

,quote,author,tags
0,“The world as we have created it is a process ...,Albert Einstein,"change, deep-thoughts, thinking, world"
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"abilities, choices"
2,“There are only two ways to live your life. On...,Albert Einstein,"inspirational, life, live, miracle, miracles"
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,"aliteracy, books, classic, humor"
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,"be-yourself, inspirational"


In [126]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   quote   100 non-null    object
 1   author  100 non-null    object
 2   tags    100 non-null    object
dtypes: object(3)
memory usage: 2.5+ KB


In [128]:
df.describe()

,quote,author,tags
count,100,100,100
unique,100,50,84
top,“The world as we have created it is a process ...,Albert Einstein,love
freq,1,10,4


In [141]:
df.to_csv('quotes.csv', index=False)